In [24]:
import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix


from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced

from sklearn import svm


In [7]:
data = pd.read_csv("./Sheet_1.csv",usecols=['response_id','class','response_text'],encoding='latin-1')
data = shuffle(data, random_state=12345)
labels = np.asarray(data['class']== 'flagged',dtype=int)


In [8]:
data.describe()

,response_id,class,response_text
count,80,80,80
unique,80,2,80
top,response_23,not_flagged,Friends often come to me to talk about the iss...
freq,1,55,1


In [9]:
data['class'].value_counts()

not_flagged    55
flagged        25
Name: class, dtype: int64

In [25]:
skf = StratifiedKFold(n_splits=5, random_state=1234, shuffle=False)
y_test_all, y_pred_all = [],[]

for i, [train_index, test_index] in enumerate(skf.split(data['response_text'],labels)):
    #print train_index, test_index;
    X_train, X_test = data['response_text'][train_index], data['response_text'][test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    pipe = make_pipeline_imb(TfidfVectorizer(),
                         RandomOverSampler(),
                         MultinomialNB())

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    y_test_all.extend(y_test)
    y_pred_all.extend(y_pred)
    print 'Fold %d' % i

print(classification_report_imbalanced(y_test_all, y_pred_all))
cnf_matrix = confusion_matrix(y_test_all, y_pred_all)

    

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
                   pre       rec       spe        f1       geo       iba       sup

          0       0.71      0.58      0.48      0.64      0.53      0.28        55
          1       0.34      0.48      0.58      0.40      0.53      0.28        25

avg / total       0.60      0.55      0.51      0.57      0.53      0.28        80



In [26]:
np.sum(np.asarray(y_test_all)==np.asarray(y_pred_all))/float(len(y_pred_all))


0.55